In [78]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

from imblearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import BayesianRidge

In [143]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

X = df_train.drop(['target', 'id'], axis=1)
y = df_train['target']

X_test_pre = df_test.drop('id', axis=1)

# Inspection of data

In [148]:
from sklearn.model_selection import train_test_split
X_train_pre, X_test_pre, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y) 

In [4]:
y_train.head()

39676    0
43345    0
13826    0
43745    0
31558    0
Name: target, dtype: int64

In [5]:
X_train_pre.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28
39676,E,AU,76123dffc,ff20ed762,6.8285,A,1.0,G,6.0,Kq,1.0,Kq,a,5bdcf7324,T,E,8.0,0.4,ead914edc,7.0,5b5ec9c40,3.0,NaN,D,S,32.0,ead914edc,j,0.0
43345,R,AU,e443234cd,c8dfa2ca5,6.8054,A,2.0,B,NaN,AI,0.0,AI,e,4abb8589e,F,R,6.0,0.5,303f4e39a,6.0,6fafa6144,2.0,1.0,F,N,27.0,303f4e39a,a,0.0
13826,Q,AU,f413bcf72,e0a06c30c,7.0033,A,1.0,G,1.0,WW,1.0,WW,a,7c77c52f3,NaN,Q,NaN,0.2,4b13d99ca,7.0,554d11173,3.0,1.0,E,N,65.0,4b13d99ca,h,0.0
43745,H,DE,9295241fe,5d7fedb99,6.9875,A,0.0,R,6.0,pB,0.0,pB,NaN,d9148f8e7,T,H,6.0,0.2,216083343,2.0,b9b1cd7f3,1.0,1.0,E,S,95.0,216083343,d,1.0
31558,F,IT,515b6eeef,c171c8cf0,7.1873,A,0.0,R,4.0,Rj,0.0,Rj,a,4f727aeee,T,F,8.0,0.5,86ec768cd,5.0,de54260a1,1.0,0.0,A,N,28.0,86ec768cd,h,0.0


In [ ]:
for elem in X:
    print(f'Column \'{elem}\': \n{X[elem].unique()}')
    print()

After inspecting each column, it looks as though column f0 and f15 are the same and f9 and f11 are the same, which also seems to be the case (see below.)

Columns f28 and f22 contains only binary values. 

Columns f2, f3, f13, f18, f20 and f26 contains hexadecimal values

Column f28 contains only binary values. 

Column f27 contains the letters from a-m.

Column f26 contains only hexadecimal values

Column f25 contains the values 1-104

Column f24 contains only "N" and "S" (North, south?)

Column f23 contains only A-F (grades?)

Column f22 contains only binary values. 

Column f21 contains the values 1-3

Column f20 contains only hexadecimal values

Column f19 contains the values 1-7

Column f18 contains only hexadecimal values

Column f17 contains the values .1 - .6

Column f16 contains the values 1-12

Column f15 contains the alphabet

Column f14 contains only T and F, which _could_ be True/False.

Column f13 contains only hexadecimal values

Column f12 contains the letters a-f (grades?) NOT THE SAME AS F23

Column f11 contains strings with two letters

Column f10 contains only the values 0, 1, 10 and 11

Column f9 contains strings with two letters

Column f8 contains the values 1-6

Column f7 contains only R, G, B (which might be Red, Green, Blue?)

Column f6 contains only the values 0-2

Column f5 contains only "A" and "B"

Column f4 contains floats

Column f3 contains only hexadecimal values

Column f2 contains only hexadecimal values

Column f1 contains only 'AU', 'CN', 'DE', 'IT', 'JP'

Column f0 contains the alphabet

In [ ]:
print('F0 == F15?', end=' ')
print(X['f0'].equals(X['f15']))

print('F9 == F11?', end=' ')
print(X['f9'].equals(X['f11']))

print('F9 == F11?', end=' ')
print(X['f12'].dropna().equals(X['f23'].dropna().apply(lambda x:x.lower())))

In [ ]:
np.sort(X['id'].dropna().unique())

In [6]:
hex_columns = ['f2', 'f3', 'f13', 'f18', 'f20', 'f26']
bool_columns = ['f28', 'f22', 'f14']
ordinal_columns = ['f6', 'f8', 'f16', 'f17', 'f19', 'f21', 'f25']
categorical_columns = ['f1', 'f5', 'f7', 'f9', 'f11', 'f12', 'f23', 'f24', 'f27']
numerical_columns = ['f4']
alphabet_columns = ['f0', 'f15']

# Duplicates

In [7]:
X.duplicated().any()

False

# Missing values

In [ ]:
round(100*(X.isnull().sum()/len(X.index)),2)

We can see that there are ~3% missing values in each column. Missing values can be handeled in many different ways, one is to drop all rows which contains missing values. This can clean up the data, but at the same time it reduces the little data we already have. Since there are only ~3% we firstly try to drop the rows, and see how much data we will miss:

In [44]:
dropped_nan = X.dropna()
print(f'Dataset percent after dropping nan-rows: {(dropped_nan.shape[0] / X.shape[0]) * 100}')
print(f'Rows removed: {X.shape[0] - dropped_nan.shape[0]}')

Dataset percent after dropping nan-rows: 46.616
Rows removed: 26692


We see that by dropping all rows with missing values, we are removing ~54% of the entire data. This is unacceptable, so we have to do something else with the missing rows. One common approach is to set the missing values as the mean of the whole column. We'll try to do this:

In [149]:
def conv_hex(x):
    try:
        return int(x, 16)
    except ValueError as e:
        return np.nan
    except TypeError as e:
        return np.nan
    
def conv_bool(x):
    try:
        if not type(x) == str:
            return x
        if x.lower() == 'f':
            return 0
        elif x.lower() == 't':
            return 1
        return x
    except e:
        return np.nan

def preprocess(df):
    for col in hex_columns:
        col_loc = list(df.columns).index(col)
        # print(df.iloc[:,col_loc])
        df.iloc[:,col_loc] = df[col].apply(lambda x: conv_hex(x)).values

    for col in bool_columns:
        col_loc = list(df.columns).index(col)
        df.iloc[:,col_loc] = df[col].apply(lambda x: conv_bool(x)).values

    for col in numerical_columns + ordinal_columns:
        df[col].fillna(np.mean(X[col]))

preprocess(X_train_pre)
preprocess(X_test_pre)
# for elem in X:
#     print(f'Column \'{elem}\': \n{X[elem].unique()}')
#     print()

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [139]:
X.shape

(50000, 29)

In [150]:
numeric_transformer = make_pipeline(
    IterativeImputer(random_state=42),
    StandardScaler()
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)
    
preprocessor = make_column_transformer( 
    (StandardScaler(), numerical_columns + hex_columns + bool_columns + ordinal_columns),
    (OneHotEncoder(), categorical_columns + alphabet_columns),
)

print(X_test_pre.shape)

X_train = pd.DataFrame(preprocessor.fit_transform(X, y).toarray())
X_test  = pd.DataFrame(preprocessor.transform(X_test_pre).toarray())

# X_train.describe()

(10000, 29)


In [124]:
X_train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498
count,4.859400e+04,4.850600e+04,4.850700e+04,4.851900e+04,4.848800e+04,4.846300e+04,4.772400e+04,4.853500e+04,4.849700e+04,4.850200e+04,4.845700e+04,4.852200e+04,4.848800e+04,4.854500e+04,4.844700e+04,4.851000e+04,4.850100e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.00000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50

# Prediction

In [127]:
X_test.shape

(10000, 499)

In [179]:
%%time

lgbm = LGBMClassifier(random_state=10, learning_rate=.125, num_leaves=20000)

lgbm.fit(X_train, y)

y_pred = lgbm.predict(X_test)

CPU times: user 1min 2s, sys: 3.84 s, total: 1min 6s
Wall time: 21.6 s


In [180]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_pred, y_test)))

LightGBM Model accuracy score: 0.8244


In [178]:
ones = 0
zeros = 0
for elem in y_pred:
    if elem == 1:
        ones += 1
    elif elem == 0:
        zeros += 1

print(f'Ones: {ones}, Zeros: {zeros}')
print(f'Zeros vs ones: {1 - ones/zeros}')

Ones: 311, Zeros: 9689
Zeros vs ones: 0.9679017442460522
